In [1]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [2]:
assert(os.environ.get("NOTION_TOKEN") is not None)
token_v2 = os.environ.get("NOTION_TOKEN")
parent_page_url_or_id = "https://www.notion.so/ezwalk/Example-usage-of-1ad592a303528014899bde92b5d12147?pvs=4"

In [3]:
from datetime import datetime

from notion.client import *
from notion.block import *
from notion.collection import NotionDate

/Users/ethanwalker/Documents/Projects/notion-py/notion/monitor.py:32: SyntaxWarning: invalid escape sequence '\d'
  for ping in re.findall('\d+:\d+"primus::ping::\d+"', thing):
/Users/ethanwalker/Documents/Projects/notion-py/notion/monitor.py:37: SyntaxWarning: invalid escape sequence '\d'
  for blob in re.findall("\d+:\d+(\{.*?\})(?=\d|$)", thing):
/Users/ethanwalker/Documents/Projects/notion-py/notion/monitor.py:188: SyntaxWarning: invalid escape sequence '\:'
  match = re.match("versions/([^\:]+):(.+)", key)


## Client Setup

In [4]:
client = NotionClient(token_v2=token_v2)
# Test the client
print(client.get_email_uid())

{'ethan@zwconsultingllc.net': 'ce4dc927-a7aa-4611-bebe-6eb7939c94ea', 'rocketwalker@gmail.com': '59164cd3-f8de-49dc-a432-e8cb48b4a39d', 'donna@zwconsultingllc.net': '133d872b-594c-810c-b916-000287c87f08'}


## Page Setup

In [61]:
parent_page = client.get_block(parent_page_url_or_id)
print(parent_page)
page = parent_page.children.add_new(
	PageBlock,
	title="Smoke test at {}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")),
)

print("Created base smoke test page at:", page.get_browseable_url())

id='1ad592a3-0352-8014-899b-de92b5d12147', title='Notion-py Smoke Tests'
Created base smoke test page at: https://www.notion.so/dbaf23bd663f4a6ebd15dede09f9e237


### Basic Blocks

In [52]:
col_list = page.children.add_new(ColumnListBlock)
col1 = col_list.children.add_new(ColumnBlock)
col2 = col_list.children.add_new(ColumnBlock)
col1kid = col1.children.add_new(
	TextBlock, title="Some formatting: *italic*, **bold**, ***both***!"
)
assert (
	col1kid.title.replace("_", "*")
	== "Some formatting: *italic*, **bold**, ***both***!"
)
assert col1kid.title_plaintext == "Some formatting: italic, bold, both!"

### To-Do Blocks

In [53]:
col2.children.add_new(TodoBlock, title="I should be unchecked")
col2.children.add_new(TodoBlock, title="I should be checked", checked=True)

<TodoBlock (id='830ab387-4a67-467c-8487-ead38a66be79', title='I should be checked', checked=True)>

### Video Blocks

In [18]:
page = client.get_block("Page URL or ID")
page.children.add_new(HeaderBlock, title="The finest music:")
video = page.children.add_new(VideoBlock, width=100)
video.set_source_url("https://www.youtube.com/watch?v=oHg5SJYRHA0")

In [19]:
assert video in page.children
assert col_list in page.children
assert video in page.children.filter(VideoBlock)
assert col_list not in page.children.filter(VideoBlock)

### Callouts

In [20]:
page.children.add_new(CalloutBlock, title="I am a callout", icon="🤞")


<CalloutBlock (id='a67e9fb1-2985-4c21-b6e7-127a3ef9abb9', title='I am a callout')>

### Mentions, Links and Backlinking

In [21]:
# check that the parent does not yet consider this page to be backlinking
assert page not in parent_page.get_backlinks()


In [22]:
page.children.add_new(SubheaderBlock, title="A link back to where I came from:")
alias = page.children.add_alias(parent_page)
assert alias.is_alias
assert not page.is_alias
page.children.add_new(
	QuoteBlock,
	title="Clicking [here]({}) should take you to the same place...".format(
		page.parent.get_browseable_url()
	),
)

<QuoteBlock (id='fe8bd6a4-a648-4e2b-853f-5c3d467fe1ed', title='Clicking [here](https://www.notion.so/1ad592a303528014899bde92b5d12147) should take you to the same place...')>

In [25]:
# check that the parent now knows about the backlink
assert page in parent_page.get_backlinks()

AssertionError: 

In [25]:
# ensure __repr__ methods are not breaking
repr(page)
repr(page.children)
for child in page.children:
	repr(child)


### Moving blocks

In [31]:
page.children.add_new(
	SubheaderBlock, title="The order of the following should be alphabetical:"
)

B = page.children.add_new(BulletedListBlock, title="B")
D = page.children.add_new(BulletedListBlock, title="D")
C2 = page.children.add_new(BulletedListBlock, title="C2")
C1 = page.children.add_new(BulletedListBlock, title="C1")
C = page.children.add_new(BulletedListBlock, title="C")
A = page.children.add_new(BulletedListBlock, title="A")

D.move_to(C, "after")
A.move_to(B, "before")
C2.move_to(C)
C1.move_to(C, "first-child")


### Collections

#### Create a collection and a view

In [28]:
def get_collection_schema():
    return {
        "%9:q": {"name": "Check Yo'self", "type": "checkbox"},
        "=d{|": {
            "name": "Tags",
            "type": "multi_select",
            "options": [
                {
                    "color": "orange",
                    "id": "79560dab-c776-43d1-9420-27f4011fcaec",
                    "value": "A",
                },
                {
                    "color": "default",
                    "id": "002c7016-ac57-413a-90a6-64afadfb0c44",
                    "value": "B",
                },
            ],
        },
        "=d{q": {
            "name": "Category",
            "type": "select",
            "options": [
                {
                    "color": "orange",
                    "id": "59560dab-c776-43d1-9420-27f4011fcaec",
                    "value": "A",
                },
                {
                    "color": "default",
                    "id": "502c7016-ac57-413a-90a6-64afadfb0c44",
                    "value": "B",
                },
            ],
        },
        "LL[(": {"name": "Person", "type": "person"},
        "4Jv$": {"name": "Estimated value", "type": "number"},
        "OBcJ": {"name": "Where to?", "type": "url"},
        "TwR:": {"name": "Some Date", "type": "date"},
        "dV$q": {"name": "Files", "type": "file"},
        "title": {"name": "Name", "type": "title"},
    }


In [29]:
cvb = page.children.add_new(CollectionViewBlock)
cvb.collection = client.get_collection(
	client.create_record("collection", parent=cvb, schema=get_collection_schema())
)
cvb.title = "My data!"
view = cvb.views.add_new(view_type="table")

special_code = uuid.uuid4().hex[:8]



#### Add a row

In [30]:
# add a row
row1 = cvb.collection.add_row()
assert row1.person == []
row1.name = "Just some data"
row1.title = "Can reference 'title' field too! " + special_code
assert row1.name == row1.title
row1.check_yo_self = True
row1.estimated_value = None
row1.estimated_value = 42
row1.files = [
	"https://www.birdlife.org/sites/default/files/styles/1600/public/slide.jpg"
]
row1.tags = None
row1.tags = []
row1.tags = ["A", "C"]
row1.where_to = "https://learningequality.org"
row1.category = "A"
row1.category = ""
row1.category = None
row1.category = "B"

start = datetime.strptime("2020-01-01 09:30", "%Y-%m-%d %H:%M")
end = datetime.strptime("2020-01-05 20:45", "%Y-%m-%d %H:%M")
timezone = "America/Los_Angeles"
reminder = {"unit": "minute", "value": 30}
row1.some_date = NotionDate(start, end=end, timezone=timezone, reminder=reminder)


#### Add another row

In [33]:
# add another row
row2 = cvb.collection.add_row(person=client.current_user, title="Metallic penguins")
assert row2.person == [client.current_user]
assert row2.name == "Metallic penguins"
row2.check_yo_self = False
row2.estimated_value = 22
row2.files = [
	"https://www.picclickimg.com/d/l400/pict/223603662103_/Vintage-Small-Monet-and-Jones-JNY-Enamel-Metallic.jpg"
]
row2.tags = ["A", "B"]
row2.where_to = "https://learningequality.org"
row2.category = "C"


#### Validate the collection

In [34]:
# check that options "C" have been added to the schema
for prop in ["=d{|", "=d{q"]:
	assert cvb.collection.get("schema.{}.options.2.value".format(prop)) == "C"


In [35]:

# check that existing options "A" haven't been affected
for prop in ["=d{|", "=d{q"]:
	assert (
		cvb.collection.get("schema.{}.options.0.id".format(prop))
		== get_collection_schema()[prop]["options"][0]["id"]
	)


#### Query a collection

In [ ]:

# Run a filtered/sorted query using the view's default parameters
result = view.default_query().execute()
assert row1 == result[0]
assert row2 == result[1]
assert len(result) == 2

In [ ]:

# query the collection directly
assert row1 in cvb.collection.get_rows(search=special_code)
assert row2 not in cvb.collection.get_rows(search=special_code)
assert row1 not in cvb.collection.get_rows(search="penguins")
assert row2 in cvb.collection.get_rows(search="penguins")


In [ ]:

# search the entire space
assert row1 in client.search_blocks(search=special_code)
assert row1 not in client.search_blocks(search="penguins")
assert row2 not in client.search_blocks(search=special_code)
assert row2 in client.search_blocks(search="penguins")


In [ ]:

# Run an "aggregation" query
aggregations = [
	{"property": "estimated_value", "aggregator": "sum", "id": "total_value"}
]
result = view.build_query(aggregations=aggregations).execute()
assert result.get_aggregate("total_value") == 64


In [ ]:

# Run a "filtered" query
filter_params = {
	"filters": [
		{
			"filter": {
				"value": {
					"type": "exact",
					"value": {"table": "notion_user", "id": client.current_user.id},
				},
				"operator": "person_does_not_contain",
			},
			"property": "person",
		}
	],
	"operator": "and",
}
result = view.build_query(filter=filter_params).execute()
assert row1 in result
assert row2 not in result


In [ ]:

# Run a "sorted" query
sort_params = [{"direction": "ascending", "property": "estimated_value"}]
result = view.build_query(sort=sort_params).execute()
assert row1 == result[1]
assert row2 == result[0]


#### Reminders and Time zones

In [41]:
# Test that reminders and time zone's work properly
row1.refresh()
assert row1.some_date.start == start
assert row1.some_date.end == end
assert row1.some_date.timezone == timezone
assert row1.some_date.reminder == reminder


## Cleanup

In [42]:
def _delete_page_fully(page):

    id = page.id

    parent_page = page.parent

    assert page.get("alive") == True
    assert page in parent_page.children
    page.remove()
    assert page.get("alive") == False
    assert page not in parent_page.children

    assert (
        page.space_info
    ), "Page {} was fully deleted prematurely, as we can't get space info about it anymore".format(
        id
    )

    page.remove(permanently=True)

    time.sleep(1)

    assert (
        not page.space_info
    ), "Page {} was not really fully deleted, as we can still get space info about it".format(
        id
    )


In [ ]:
_delete_page_fully(page)

# Basic AI completions

In [5]:
# Use this block to monitor an agent description page for backlinks, 
# which are automatically created when the agent page is @ mentioned

# Set the url to a page 
url_of_agent_page = "https://www.notion.so/ezwalk/Sample-Agent-3aef4872756e4d6380e5e3926ec33c71?pvs=4"
agent_page = client.get_block(url_of_agent_page)
agent_backlinks = []

In [ ]:
# poll the message store for new backlinks
while True:
	for link in agent_page.get_backlinks():
		if link in agent_backlinks:
			link_location = agent_backlinks.index(link)
			agent_backlinks.insert(link_location, link)
			agent_backlinks.pop(link_location+1)
	print(agent_backlinks)
	time.sleep(1)

In [12]:
agent_backlinks[0].__dict__

IndexError: list index out of range

In [7]:
request_queue = []

In [15]:
# populate the request Queue
try:
	last_backlink = agent_page.get_backlinks()[0]
	if last_backlink.id not in request_queue:
		request_queue.append(
			{
				"id": last_backlink.id,
				"status": "queued",
				"prompt":last_backlink.title_plaintext
			}	
      	)
	print(request_queue)
except Exception as e:
    raise e

[{'id': '1bd592a3-0352-80cf-b292-e62341b2e6fd', 'status': 'queued', 'prompt': 'Sample Agent '}, {'id': '1bd592a3-0352-8014-a747-dedaface5076', 'status': 'queued', 'prompt': 'Sample Agent How about a third mention'}, {'id': '1bd592a3-0352-80cf-b292-e62341b2e6fd', 'status': 'queued', 'prompt': 'Sample Agent '}]


In [36]:
request_queue

[{'id': '1b6592a3-0352-80d6-a19e-d6b8e3383bb9',
  'status': 'queued',
  'prompt': 'Sample Agent Tell me again why the earth is flat'}]

In [12]:
from openai import OpenAI
openai_client = OpenAI()

completion = openai_client.chat.completions.create(
    model="gpt-4o",
    messages=[{
        "role": "user",
        "content": "Write a one-sentence bedtime story about a unicorn."
    }]
)

In [37]:
print(completion.choices[0].message.content)

The idea that the Earth is flat is a misconception that has been conclusively debunked by science. The Earth is, in fact, an oblate spheroid, which means it is mostly spherical but slightly flattened at the poles and bulging at the equator due to its rotation.

Here are some key points that support the understanding that the Earth is round:

1. **Historical Evidence**: Ancient Greek philosophers and mathematicians, such as Pythagoras and Eratosthenes, provided early evidence of a spherical Earth. Eratosthenes, for example, calculated the circumference of the Earth with remarkable accuracy over 2,000 years ago.

2. **Photographic Evidence**: Since the advent of space travel in the 20th century, photographs taken from space have consistently shown the Earth as a round object.

3. **Physical Phenomena**: Observations such as the way ships disappear hull-first over the horizon, the circular shadow of the Earth on the Moon during a lunar eclipse, and the way constellations change with latit

In [38]:
for request in request_queue:
    print(request['id'])
    if request['status'] == "queued":
        # Do AI call
        completion = openai_client.chat.completions.create(
			model="gpt-4o",
			messages=[{
				"role": "user",
				"content": request["prompt"]
			}]
		)
        
        # Get the calling block
        request_block = client.get_block(request['id'])
        
        # Add the completion response as a child to the calling block
        request_block.children.add_new(TextBlock, title=completion.choices[0].message.content)
    

1b6592a3-0352-80d6-a19e-d6b8e3383bb9
